In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn import metrics
from math import sqrt
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import sys
import datetime 
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn import metrics
from math import sqrt
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import sys
import datetime

In [13]:
class Classifier:
    
    def __init__(self):
        pass
    
    def file_input(self,filename):
        data = pd.read_csv(filename,engine='python',header=None)
        return data
    
    def my_function(self,list1,list2,list3):
        if list3 == 1:
            if list1 in list2:
                return 1
            else:
                return 0
            
    def convertCategory(self,x):
        if x == 'S':
            return -1
        elif x in ['1','2','3','4','5','6','7','8','9','10','11','12']:
            return x
        else:
            return 13
    
    def data_exploration_clicks(self,clicks):
        
        category_analaysis = clicks[(clicks['category'].isin(['S','0','1','2','3','4','5','6','7','8','9','10','11','12']))]
      
        chart_4 = sns.barplot(x=category_analaysis['category'].value_counts().index, y=category_analaysis['category'].value_counts())
        chart_4.set_xticklabels(chart_4.get_xticklabels(),rotation=45)
        plt.xlabel('Category')
        plt.ylabel('Count')
        plt.title('Count of clicks against each Category')
        plt.show()
   
        chart_5 = sns.barplot(x=clicks['item_id'].value_counts().nlargest(10).index, y=clicks['item_id'].value_counts().nlargest(10))
        chart_5.set_xticklabels(chart_5.get_xticklabels(),rotation=45)
        plt.xlabel('Item_id')
        plt.ylabel('Count')
        plt.title('Top 10 Items having maximum clicks:')
        plt.show()
        
    def data_exploration_buys(self,data):
        
        print("Top 10 Items which have been bought the maximum.")
        #print(buys['item_id'].value_counts().nlargest(10))
        chart_1 = sns.barplot(x=buys['item_id'].value_counts().nlargest(10).index, y=buys['item_id'].value_counts().nlargest(10))
        chart_1.set_xticklabels(chart_1.get_xticklabels(),rotation=45)
        plt.xlabel('Item_id')
        plt.ylabel('Count')
        plt.title('Top 10 Items which have been bought the maximum.')
        plt.show()
        
        print("Top 10 items which are purchased in larger quantities.")
        quantity_analysis = buys[['item_id','qty']].groupby('item_id').agg(total_quantity=pd.NamedAgg(column='qty',aggfunc=sum))
        quant_analysis = quantity_analysis.sort_values('total_quantity',ascending=False).nlargest(10,columns='total_quantity')
        chart_2 = sns.barplot(x = quant_analysis.index, y = quant_analysis['total_quantity'] ,data = quant_analysis)
        chart_2.set_xticklabels(chart_2.get_xticklabels(),rotation=45)
        plt.xlabel('Item_id')
        plt.ylabel('Quantity')
        plt.title('Top 10 items which are purchased in larger quantities.')
        plt.show()
        
        print("Top 10 items Identifying the items having the maximum price.")
        buys_plot = buys[['item_id','price']].drop_duplicates().sort_values('price',ascending=False).nlargest(10,columns='price')
        chart_3 = sns.barplot(x = buys_plot['item_id'] , y = buys_plot['price'] ,data = buys_plot)
        chart_3.set_xticklabels(chart_3.get_xticklabels(),rotation=45)
        plt.xlabel('Item_id')
        plt.ylabel('Price')
        plt.title('Top 10 items having the maximum cost price.')
        plt.show()
        
    def transforming_buys(self,buys):
        print("Transforming the buys file ...!!!")
        grouped = buys.groupby("session")
        buys_g = pd.DataFrame(index=grouped.groups.keys())        
        buys_g["Number_items_bought"] = grouped.item_id.count()
        buys_g["unique_items_bought"] = grouped.item_id.unique()
        buys_g["is_buy"] = 1
        buys_g.index.name = "session"
        print("Transformation of the buys file completed...!!!")
        return buys_g
    
    def chunk_load_data(self,chunk):
        return chunk
        
        
    def transforming_clicks(self,clicks):
        
        clicks_new = clicks.groupby('session')['timestamp'].agg([min,max])

        clicks_new['dwell_time'] = clicks_new['max'] - clicks_new['min'] #cal the dwell time of the session.
        clicks_new['dwell_time_seconds'] = clicks_new['dwell_time'].dt.total_seconds() #converting dwell time into seconds
        
        clicks.loc[clicks['category'] == 'S',['category']] = -1

        grouped = clicks.groupby('session')
            
        #print("Calculating the total clicks")
        clicks_new['total_clicks'] = grouped.item_id.count()
        
        #print("Calculating the day of week")
        clicks_new['dayofweek'] = clicks_new['min'].dt.dayofweek
        
        #print("Calculating the day of month")
        clicks_new['dayofmonth'] = clicks_new['min'].dt.day
        
        #print("Calculating hour of click")
        clicks_new['hourofclick'] = clicks_new['min'].dt.hour
        
        #print("Calculating time of click")
        b = [0,4,8,12,16,20,24]
        l = ['Late Night', 'Early Morning','Morning','Noon','Evening','Night']
        clicks_new['timeofday'] = pd.cut(clicks_new['hourofclick'], bins=b, labels=l, include_lowest=True)
        
        #print("Calculating clickrate")
        clicks_new["click_rate"] = clicks_new["total_clicks"] / clicks_new["dwell_time_seconds"]
        clicks_new.click_rate = clicks_new.click_rate.replace(np.inf, np.nan)
        clicks_new.click_rate = clicks_new.click_rate.fillna(0)
        
        #print("** Transformed**")
        return clicks_new

    def transforming_clicks2(self,clicks):

        grouped = clicks.groupby('session').agg({'item_id':['first','last','nunique'],'category':['nunique']})        
        return grouped

    def transforming_clicks3(self,clicks):
       
        keys, values = clicks.sort_values('session').values.T
        ukeys, index = np.unique(keys, True)
        arrays = np.split(values, index[1:])
        df2 = pd.DataFrame({'a':ukeys, 'b':(a for a in arrays)})
        return df2
    
    def transforming_clicks3_cat(self,clicks):
       
        keys, values = clicks.sort_values('session').values.T
        ukeys, index = np.unique(keys, True)
        arrays = np.split(values, index[1:])
        df2 = pd.DataFrame({'a':ukeys, 'b':(a for a in arrays)})
        return df2

    def data_preparation(self,X,y):
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
        
        return X_train, X_test, y_train, y_test
    
        
    def undersampling(self,train_data):   
        
        count_class_0, count_class_1 = train_data['is_buy'].value_counts()
        df_class_0 = train_data[train_data['is_buy'] == 0]
        df_class_1 = train_data[train_data['is_buy'] == 1]
        df_class_0_under = df_class_0.sample(count_class_1)
        df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)
        df_test_under['is_buy'].value_counts()
        return df_test_under
        
    def one_hot_encode(self,column_name,training_data):
        temp = pd.get_dummies(training_data[column_name])
        training_data = pd.concat([training_data, temp], axis=1)
        return training_data
        
    def get_preds(self,threshold, probabilities):
        return [1 if prob > threshold else 0 for prob in probabilities]
        
    def logit_model(self,train_x,train_y,test_x,test_y,thres=0.5):
        
        model = LogisticRegression(solver='sag')
        model.fit(train_x,train_y.values.ravel())
        probas = model.predict_proba(test_x)[:, 1]
        print("Threshold Value : ",thres)
        y_pred = self.get_preds(thres,probas)
        return y_pred,probas

    def calc_special_offer(self,x):
        if -1 in x:
            return 1
        else:
            return 0
    
    def error_metrics(self,prediction,test_y,probas):
        
        accuracy = accuracy_score(prediction,test_y)        
        print('Accuracy =',accuracy)
        print("")
        print(pd.DataFrame(confusion_matrix(test_y, prediction), columns=['Predicted 0', "Predicted 1"], index=['Actual 0', 'Actual 1']))
        print("classification_Report:")
        print(classification_report(test_y,prediction))
        fig, ax = plt.subplots(figsize=(10,7))
        fpr, tpr, threshold = metrics.roc_curve(test_y,probas,pos_label=1)
        i = np.arange(len(tpr)) 
        roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
        roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]
        print("Optimum Threshold Value:",list(roc_t['threshold']))
        
        auc = metrics.roc_auc_score(test_y, probas)
        plt.plot(fpr,tpr,label="auc="+str(auc))
        ax.plot(np.linspace(0, 1, 100),np.linspace(0, 1, 100),label='baseline',linestyle='--')
        plt.title('Receiver Operating Characteristic Curve', fontsize=18)
        plt.ylabel('TPR', fontsize=16)
        plt.xlabel('FPR', fontsize=16)
        plt.legend(fontsize=12);
        plt.show()

In [14]:
!ls

train.ipynb train.py


In [15]:
clicks_file = "../../data/yoochoose-clicks.dat"
buys_file = "../../data/yoochoose-buys.dat"

In [16]:
data = Classifier()
result=None
count=1
names=["session", "timestamp", "item_id", "category"]
for chunk in pd.read_csv(clicks_file,names=names,usecols = ['session','timestamp','item_id','category'],parse_dates=["timestamp"],chunksize=500000):
    print("Executing Chunk ",count,"/67")
    click_df = data.transforming_clicks(chunk)
    if result is None:
      result=click_df
      count=count+1
    else:
      result = result.append(click_df)  
      count=count+1
print("Done 1st Transformation of Clicks file to calculate \n1.Session Start Time \n2.Session End Time  \n3.Session Dwell Time \n4.Dwell time seconds \n5.Total clicks \n6.Dayofweek \n7.Dayofmonth \n8.Hourofclick \n9.Timeofday \n10.Click_Rate")

Executing Chunk  1 /67
Executing Chunk  2 /67
Executing Chunk  3 /67
Executing Chunk  4 /67
Executing Chunk  5 /67
Executing Chunk  6 /67
Executing Chunk  7 /67
Executing Chunk  8 /67
Executing Chunk  9 /67
Executing Chunk  10 /67
Executing Chunk  11 /67
Executing Chunk  12 /67
Executing Chunk  13 /67
Executing Chunk  14 /67
Executing Chunk  15 /67
Executing Chunk  16 /67
Executing Chunk  17 /67
Executing Chunk  18 /67
Executing Chunk  19 /67
Executing Chunk  20 /67
Executing Chunk  21 /67
Executing Chunk  22 /67
Executing Chunk  23 /67
Executing Chunk  24 /67
Executing Chunk  25 /67
Executing Chunk  26 /67
Executing Chunk  27 /67
Executing Chunk  28 /67
Executing Chunk  29 /67


/Users/pratikkasle/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Executing Chunk  30 /67
Executing Chunk  31 /67
Executing Chunk  32 /67
Executing Chunk  33 /67
Executing Chunk  34 /67
Executing Chunk  35 /67
Executing Chunk  36 /67
Executing Chunk  37 /67
Executing Chunk  38 /67
Executing Chunk  39 /67
Executing Chunk  40 /67
Executing Chunk  41 /67
Executing Chunk  42 /67
Executing Chunk  43 /67
Executing Chunk  44 /67
Executing Chunk  45 /67
Executing Chunk  46 /67
Executing Chunk  47 /67
Executing Chunk  48 /67
Executing Chunk  49 /67
Executing Chunk  50 /67
Executing Chunk  51 /67
Executing Chunk  52 /67
Executing Chunk  53 /67
Executing Chunk  54 /67
Executing Chunk  55 /67
Executing Chunk  56 /67
Executing Chunk  57 /67
Executing Chunk  58 /67
Executing Chunk  59 /67
Executing Chunk  60 /67
Executing Chunk  61 /67
Executing Chunk  62 /67
Executing Chunk  63 /67
Executing Chunk  64 /67
Executing Chunk  65 /67
Executing Chunk  66 /67
Executing Chunk  67 /67
Done 1st Transformation of Clicks file to calculate 
1.Session Start Time 
2.Session End

In [17]:
result_2=None
count=1
names=["session", "timestamp", "item_id", "category"]
for chunk in pd.read_csv(clicks_file,usecols = ['session','item_id','category'],names=names,chunksize=500000):
    print("Executing Chunk ",count,"/67")
    click_df = data.transforming_clicks2(chunk)
    if result_2 is None:
      result_2=click_df
      count=count + 1
    else:
      result_2 = result_2.append(click_df) 
      count= count + 1
print("Done Transforming Clicks input file to calculate \n 1.First Clicked item \n 2.Last Clicked Item \n 3.Total Unique Items  \n 4.Total Unique Categories ")
colnames=['first_clicked_item','last_clicked_item','total_unique_items','total_unique_categories']
result_2.columns = colnames
#result_2.set_index('session')

Executing Chunk  1 /67
Executing Chunk  2 /67
Executing Chunk  3 /67
Executing Chunk  4 /67
Executing Chunk  5 /67
Executing Chunk  6 /67
Executing Chunk  7 /67
Executing Chunk  8 /67
Executing Chunk  9 /67
Executing Chunk  10 /67
Executing Chunk  11 /67
Executing Chunk  12 /67
Executing Chunk  13 /67
Executing Chunk  14 /67
Executing Chunk  15 /67
Executing Chunk  16 /67
Executing Chunk  17 /67
Executing Chunk  18 /67
Executing Chunk  19 /67
Executing Chunk  20 /67
Executing Chunk  21 /67
Executing Chunk  22 /67
Executing Chunk  23 /67
Executing Chunk  24 /67
Executing Chunk  25 /67
Executing Chunk  26 /67
Executing Chunk  27 /67
Executing Chunk  28 /67
Executing Chunk  29 /67


/Users/pratikkasle/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Executing Chunk  30 /67
Executing Chunk  31 /67
Executing Chunk  32 /67
Executing Chunk  33 /67
Executing Chunk  34 /67
Executing Chunk  35 /67
Executing Chunk  36 /67
Executing Chunk  37 /67
Executing Chunk  38 /67
Executing Chunk  39 /67
Executing Chunk  40 /67
Executing Chunk  41 /67
Executing Chunk  42 /67
Executing Chunk  43 /67
Executing Chunk  44 /67
Executing Chunk  45 /67
Executing Chunk  46 /67
Executing Chunk  47 /67
Executing Chunk  48 /67
Executing Chunk  49 /67
Executing Chunk  50 /67
Executing Chunk  51 /67
Executing Chunk  52 /67
Executing Chunk  53 /67
Executing Chunk  54 /67
Executing Chunk  55 /67
Executing Chunk  56 /67


In [ ]:
result_3=None
count=1
names=["session", "timestamp", "item_id", "category"]
for chunk in pd.read_csv(clicks_file,names=names,usecols = ['session','item_id'],chunksize=500000):
    print("Executing Chunk ",count,"/67")
    click_df = data.transforming_clicks3(chunk)
    if result_3 is None:
      result_3=click_df
      count = count + 1
    else:
      result_3 = result_3.append(click_df) 
      count = count + 1
    
print("Done Transforming Clicks")
colnames=['session','visited_items']
result_3.columns = colnames
result_3 = result_3.set_index('session')
print(datetime.datetime.now())

In [ ]:
result_4=None
count=1
names=["session", "timestamp", "item_id", "category"]
for chunk in pd.read_csv(clicks_file,names=names,usecols = ['session','category'],converters={"category": data.convertCategory},chunksize=500000):
    print("Executing Chunk ",count,"/67")
    click_df = data.transforming_clicks3_cat(chunk)
    if result_4 is None:
      result_4 = click_df
      count = count + 1
    else:
      result_4 = result_4.append(click_df)  
      count = count + 1
colnames=['session','visited_categories']
result_4.columns = colnames
result_4 = result_4.set_index('session')
result_4['Number_clicked_visited_categories'] = result_4['visited_categories'].apply(lambda x : len(x))
result_4['Special_offer_click']=result_4['visited_categories'].apply(data.calc_special_offer)
print("Done Transforming Clicks to calculate \n 1.Unique Visited categories \n 2. Total Visited Categories \n 3.Special offer click " )

In [ ]:
buys = data.file_input(buys_file)
names=["session","timestamp","item_id","price","qty"]
buys.columns = names
data.data_exploration_buys(buys)

In [ ]:
buys_g = data.transforming_buys(buys)
clicks_tranformed_updated = pd.concat([result,result_2,result_3,result_4], axis=1)
training_data = pd.merge(clicks_tranformed_updated,buys_g['is_buy'],how='left',left_index=True,right_index=True)
training_data['is_buy'] = training_data['is_buy'].fillna(0)
training_data.head()

In [ ]:
#Calculating the popularity index for first and last item clicked
print("calculating the popularity index for first and last item clicked")
names=["session", "timestamp", "item_id", "category"]
result_items = pd.concat([ chunk.apply(pd.Series.value_counts) for chunk in pd.read_csv(clicks_file,names=names,usecols = ['item_id'],index_col=0,chunksize=500000)])
df = pd.DataFrame(result_items.index.value_counts())
df.index.name = "item_id"
df.columns = ['count']
val = df['count'].sum()
df['popularity'] = df['count'].apply(lambda x : x / val )
df['popularity'] = df['popularity'].round(5)
print("Done..!!")